In [ ]:
#!pip install tensorflow==2.9.1
!pip install keras
!pip install Sentencepiece
!pip install torch
!pip install transformers==4.17
!pip install datasets
!pip install -U scikit-learn
!pip install -U sentence-transformers

In [2]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

2023-09-14 07:47:33.181662: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-14 07:47:33.311791: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-14 07:47:33.937172: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-09-14 07:47:33.937302: W tensorflow/

In [9]:
transcripts = pd.read_csv("turns_with_context.csv", index_col="Unnamed: 0")
transcripts = transcripts.fillna("")
transcripts.head()

,condition,chat_id,user_id,target_behaviour,readiness_to_change_start,utterance_id,turn_id,datetime,user_utterance,valence,...,label,label_score,sublabel,sublabel_score,bot_action,bot_datetime,bot_utterance,user_rating,user_rating_explanation,user_word_count
0,framework,62,5592a7f2fdf99b3a9e651fd9,gesünder essen,7,699,1,2023-08-08 15:04:05.82962,start,,...,,0.000000,,0.0,,2023-08-08 15:04:07.650432,"Hallo! Ich bin MIcha, dein Motivations-Chatbot...",,,1
1,framework,62,5592a7f2fdf99b3a9e651fd9,gesünder essen,7,701,2,2023-08-08 15:04:44.162145,Ich möchte mich gesünder ernähren,,...,,0.000000,,0.0,,2023-08-08 15:04:46.015272,Du möchtest also gesünder essen. Kannst du mir...,,,5
2,framework,62,5592a7f2fdf99b3a9e651fd9,gesünder essen,7,705,3,2023-08-08 15:06:12.220701,Ich möchte abnehmen und denke gesündere Ernähr...,,...,,0.000000,,0.0,,2023-08-08 15:06:15.721791,Wie zuversichtlich bist du auf einer Skala von...,,,10
3,framework,62,5592a7f2fdf99b3a9e651fd9,gesünder essen,7,710,4,2023-08-08 15:08:06.196274,4,,...,,0.000000,,0.0,,2023-08-08 15:08:08.043497,Eine 4. Das klingt als stündest du dieser Verä...,,,1
4,framework,62,5592a7f2fdf99b3a9e651fd9,gesünder essen,7,716,5,2023-08-08 15:09:14.610184,Ich bin noch unsicher ob der Zeitpunkt gut ist.,sustain,...,Reason,0.998911,General Reason,0.99149,Continuing the Paragraph,2023-08-08 15:09:23.14756,"Du bist dir unsicher, ob der Zeitpunkt gut ist...",gut/hilfreich,Ich bin beruflich sehr eingespannt.,9


In [10]:
transcripts.columns

Index(['condition', 'chat_id', 'user_id', 'target_behaviour',
       'readiness_to_change_start', 'utterance_id', 'turn_id', 'datetime',
       'user_utterance', 'valence', 'valence_score', 'label', 'label_score',
       'sublabel', 'sublabel_score', 'bot_action', 'bot_datetime',
       'bot_utterance', 'user_rating', 'user_rating_explanation',
       'user_word_count'],
      dtype='object')

In [11]:
translated_transcripts = transcripts[["chat_id", "turn_id"]].copy()

In [12]:
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-de-en")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-de-en")

In [13]:
pipe = pipeline("text2text-generation", tokenizer=tokenizer, model=model, device=0)
user_origs = list(transcripts["user_utterance"])
bot_origs = list(transcripts["bot_utterance"])
user_translations = []
bot_translations = []
for line in user_origs:
    user_trans = pipe(line)
    user_translations.append(user_trans[0]["generated_text"])

for line in bot_origs:
    bot_trans = pipe(line)
    bot_translations.append(bot_trans[0]["generated_text"])
    
translated_transcripts["seeker_post"] = user_translations
translated_transcripts["response_post"] = bot_translations

/opt/conda/lib/python3.9/site-packages/transformers/pipelines/base.py:997: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [14]:
translated_transcripts.head()

,chat_id,turn_id,seeker_post,response_post
0,62,1,Start,"Hi! I'm MIcha, your motivational chatbot. My g..."
1,62,2,I want to eat healthier,So you want to eat healthier. Can you tell me ...
2,62,3,I want to lose weight and think healthier diet...,How confident are you on a scale from 0 (not a...
3,62,4,4,A 4. That sounds like you're not too confident...
4,62,5,I'm still not sure if the time is good.,You're not sure if the time is good. Is there ...


In [15]:
translated_transcripts.to_csv("translated_transcripts.csv")